In [ ]:
# Tutorials used:
# - https://nextjournal.com/gkoehler/pytorch-mnist
# - https://medium.com/@nutanbhogendrasharma/pytorch-convolutional-neural-network-with-mnist-dataset-4e8a4265e118

In [2]:
# Set constants
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 1000 # I don't think this actually matters

NUMBER_OF_EPOCHS = 25
LEARNING_RATE = 0.05
MOMENTUM = 0

USE_PNG_FORMAT = False

# The accuracy is measured on the test data, not the train one
# With MOMENTUM=0, LEARNING_RATE=0.01 and BATCH_SIZE=32 I got 76% accuracy after 25 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.02 and BATCH_SIZE=32 I got 68% accuracy after 9 EPOCHS, 85% accuracy after 25 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.05 and BATCH_SIZE=32 I got 78% accuracy after 25 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.1  and BATCH_SIZE=32 I got 70% accuracy after 25 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.05 and BATCH_SIZE=32 I got 88% accuracy after 25 EPOCHS, with Dropout(p=0.2) at the end of conv1 and conv2, 89 after 29 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.05 and BATCH_SIZE=32 I got 88% accuracy after 11 EPOCHS, with Dropout(p=0.5) at the beginning of conv1 + Dropout(p=0.2) aththe end of conv2, LeakyReLU from conv2 removed
# With MOMENTUM=0, LEARNING_RATE=0.05 and BATCH_SIZE=32 I got 88% accuracy after 9 EPOCHS, with Dropout(p=0.2) at the beginning of conv1 + Dropout(p=0.2) aththe end of conv2, LeakyReLU from conv2 removed
# With MOMENTUM=0, LEARNING_RATE=0.05 and BATCH_SIZE=32 I got _% accuracy after _ EPOCHS, with Dropout2D(p=0.2) at the beginning of conv1 + Dropout2D(p=0.2) aththe end of conv2, LeakyReLU from conv2 removed

In [3]:
import exercise2_config as config
import torch
import numpy as np
import pandas as pd
import os
from datetime import datetime
from PIL import Image


torch.backends.cudnn.enabled = False
torch.manual_seed(round(datetime.timestamp(datetime.now())))


def ReadImages(dir):
  images = []
  for filename in os.listdir(dir):
    image_path = os.path.join(dir, filename)
    image = Image.open(image_path)
    image_as_array = np.array(image.getdata())
    images.append(image_as_array)
    return images


def BuildDataLoadersFromPngFiles():
  for label in range(10):
    dir = os.path.join(config.PNG_TRAIN_DATA_DIR, f'{label}')
    images = ReadImages(dir)
    print(images[0])
    input()

  pass


def BuildDataLoadersFromCsvFiles():
  train_images = pd.read_csv(config.TRAIN_DATA_FILE, header=None)
  test_images = pd.read_csv(config.TEST_DATA_FILE, header=None)

  train_data = np.array(train_images.iloc[:,1:])
  train_labels = np.array(train_images.iloc[:,0])
  test_data = np.array(test_images.iloc[:,1:])
  test_labels = np.array(test_images.iloc[:,0])

  train_data = np.reshape(train_data, (len(train_data), 1, 28, 28))
  # Normalize the data
  train_data = torch.from_numpy(train_data).float()/255
  train_labels = torch.from_numpy(np.array(train_labels))
  train_set = torch.utils.data.TensorDataset(train_data, train_labels)

  test_data = np.reshape(test_data, (len(test_data), 1, 28, 28))
  # Normalize the data
  test_data = torch.from_numpy(test_data).float()/255
  test_labels = torch.from_numpy(np.array(test_labels))
  test_set = torch.utils.data.TensorDataset(test_data, test_labels)
  
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE_TEST, shuffle=True)
  return train_loader, test_loader


# We first import the data and convert them to a torch version
def BuildDataLoaders(use_png_format=USE_PNG_FORMAT):
  if use_png_format:
    return BuildDataLoadersFromPngFiles()
  return BuildDataLoadersFromCsvFiles()


train_loader, test_loader = BuildDataLoaders()


In [16]:
%load_ext autoreload
%autoreload 2
import model_task2c
import torch.nn.functional as F


# Something is not quite right and the values become nan
torch.autograd.set_detect_anomaly(True) 


# Initialize classifier stuff
cnn_model = model_task2c.PR_CNN()   # initialization of the model
optimizer = torch.optim.SGD(cnn_model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

print(cnn_model)


# Compute the accuracy of the model on the given data set
def ComputeAccuracy(loader, loss_func='cross_entropy'):
    cnn_model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        for _, (data, label) in enumerate(loader):
            output = cnn_model(data)
            
            if loss_func == 'cross_entropy':
                total_loss += F.cross_entropy(output, label, size_average=False).item()
            else:
                total_loss += F.nll_loss(output, label, size_average=False).item()

            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).sum()

    
    average_loss = total_loss / len(loader.dataset)
    print('Accuracy: {}/{} ({:.2f}%), Avg. loss: {:.4f}, \n'.format(
        correct, 
        len(loader.dataset), 
        100. * correct / len(loader.dataset),
        average_loss))


def train(loss_func='cross_entropy'):
    loss_per_epoch = []

    for epoch in range(NUMBER_OF_EPOCHS):
        print(f'Epoch {epoch + 1}')

        cnn_model.train()  # activation of the train mode

        for _, (data, label) in enumerate(train_loader):
            # Set gradients to 0. PyTorch accumulates the gradients 
            # https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
            optimizer.zero_grad()

            # forward + backward + optimize
            output = cnn_model(data)

            if loss_func == 'cross_entropy':
                loss = F.cross_entropy(output, label)
            else:
                loss = F.nll_loss(output, label)
            loss_per_epoch.append(loss)

            loss.backward()
            optimizer.step()
        
        # Compute the accuracy of the model at the end of each epoch
        ComputeAccuracy(train_loader, loss_func)

    return loss_per_epoch

loss_per_epoch = train()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PR_CNN(
  (conv1): Sequential(
    (0): Dropout2d(p=0.2, inplace=False)
    (1): Conv2d(1, 8, kernel_size=(3, 3), stride=(3, 3))
    (2): LeakyReLU(negative_slope=0.01)
  )
  (conv2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): Dropout2d(p=0.2, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 384, kernel_size=(2, 2), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (fc): Sequential(
    (0): Flatten()
    (1): Linear(in_features=1536, out_features=10, bias=True)
  )
)
Epoch 1


/home/adriana/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Accuracy: 49824/60000 (83.04%), Avg. loss: 1.6397, 

Epoch 2

Accuracy: 51054/60000 (85.09%), Avg. loss: 1.6141, 

Epoch 3

Accuracy: 51460/60000 (85.77%), Avg. loss: 1.6056, 

Epoch 4

Accuracy: 51916/60000 (86.53%), Avg. loss: 1.5979, 

Epoch 5

Accuracy: 52120/60000 (86.87%), Avg. loss: 1.5939, 

Epoch 6

Accuracy: 52550/60000 (87.58%), Avg. loss: 1.5872, 

Epoch 7

Accuracy: 52588/60000 (87.65%), Avg. loss: 1.5860, 

Epoch 8

Accuracy: 52794/60000 (87.99%), Avg. loss: 1.5821, 

Epoch 9

Accuracy: 52891/60000 (88.15%), Avg. loss: 1.5804, 

Epoch 10

Accuracy: 52950/60000 (88.25%), Avg. loss: 1.5796, 

Epoch 11

Accuracy: 53024/60000 (88.37%), Avg. loss: 1.5780, 

Epoch 12

Accuracy: 53162/60000 (88.60%), Avg. loss: 1.5758, 

Epoch 13

Accuracy: 53105/60000 (88.51%), Avg. loss: 1.5765, 

Epoch 14

Accuracy: 53269/60000 (88.78%), Avg. loss: 1.5735, 

Epoch 15

Accuracy: 53225/60000 (88.71%), Avg. loss: 1.5739, 

Epoch 16


In [ ]:
# Compute the accuracy of the model on the test dataset
ComputeAccuracy(test_loader)